In [231]:
import pandas as pd

In [232]:
df = pd.read_csv('../data/mock_drug_inventory.csv', index_col=0)

In [ ]:
def adjust_prices(df: pd.DataFrame) -> pd.DataFrame:
    required = {'base_stock', 'current_stock', 'price'}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f'Missing required columns: {sorted(missing)}')


    df = df.copy()

    # Define diff so positive => understock (base > current)
    diff = df['base_stock'] - df['current_stock']

    mask_under = diff > 0
    mask_over  = diff < 0
    # equal is implicit

    # price sanity
    df['price'] = pd.to_numeric(df['price'], errors='coerce')
    if df['price'].isna().any():
        raise ValueError('Non-numeric prices present.')
    if (df['price'] < 0).any():
        raise ValueError('Negative prices detected.')


    # Rates with caps + no negatives
    df['rate_increase'] = (0.02 * diff).clip(lower=0, upper=0.5)
    df['rate_decrease'] = (0.015 * (-diff)).clip(lower=0, upper=0.3)

    # Start from original price
    df['adjusted_price'] = df['price']

    # Apply conditionally
    df.loc[mask_under, 'adjusted_price'] = (
        df.loc[mask_under, 'price'] * (1 + df.loc[mask_under, 'rate_increase'])
    )
    df.loc[mask_over, 'adjusted_price'] = (
        df.loc[mask_over, 'price'] * (1 - df.loc[mask_over, 'rate_decrease'])
    )

    # Round for currency
    df['adjusted_price'] = df['adjusted_price'].round(2)

    return df

In [234]:
# equal → no change
df_eq = pd.DataFrame({'base_stock':[100], 'current_stock':[100], 'price':[10.0]})
assert adjust_prices(df_eq)['adjusted_price'].iloc[0] == 10.00

# under → increase, capped at 50%
df_under = pd.DataFrame({'base_stock':[200], 'current_stock':[150], 'price':[100.0]})
# 50 diff * 0.02 = 1.00 → cap 0.5 → 150.00
assert adjust_prices(df_under)['adjusted_price'].iloc[0] == 150.00

# over → decrease, capped at 30%
df_over = pd.DataFrame({'base_stock':[100], 'current_stock':[130], 'price':[100.0]})
# 30 diff * 0.015 = 0.45 → cap 0.3 → 70.00
assert adjust_prices(df_over)['adjusted_price'].iloc[0] == 70.00

In [235]:
adjusted = adjust_prices(df)
adjusted.to_csv('../data/mock_drug_inventory_adjusted.csv', index=False)